## Connecting Colab to google drive

Run the below cells to connect a google drive folder to your colab instance


In [ ]:
# you will be prompted with a window asking to grant permissions
# click connect to google drive, choose your account, and click allow
from google.colab import drive

drive.mount("/content/drive", force_remount=True)

In [ ]:
# TODO: fill in the path in your Google Drive in the string below
# Note: do not escape slashes or spaces in the path string
import os

os.chdir(
    "/content/drive/MyDrive/mp3"
)  # fill in with the path to the google drive folder where your mp is.

In [ ]:
# reload imports when running a function(this will make it so you don't have to reload the entire notebook if you change a function that you are importing and want the change to register)
%reload_ext autoreload
%autoreload 2


In [ ]:
!pip install -r requirements.txt

In [ ]:
import torch
from torch import nn
import torchvision
from torch.utils.data import DataLoader
from network import RetinaNet
from detection_utils import compute_targets, get_detections, set_seed
from tensorboardX import SummaryWriter
from absl import app, flags
from tqdm import tqdm
import numpy as np
from dataset import CocoDataset, Resizer, Normalizer, collater
from torchvision import transforms
import losses

In [ ]:
lr = 1e-3
momentum = 0.9
weight_decay = 1e-4
output_dir = "runs/retina-net-basic/"
batch_size = 1
seed = 2
max_iter = 120000
lr_step = [60000, 100000]

## define model, Loss Function, dataloader, and learning rate scheduler


In [ ]:
torch.manual_seed(seed)
set_seed(seed)
dataset_train = CocoDataset('train', seed=seed,
    transform=transforms.Compose([Normalizer(), Resizer()]))
dataset_val = CocoDataset('val', seed=0,
    transform=transforms.Compose([Normalizer(), Resizer()]))
dataloader_train = DataLoader(dataset_train, num_workers=3, collate_fn=collater)

model = RetinaNet(p67=True, fpn=True)

num_classes = dataset_train.num_classes
device = torch.device('cuda:0')
model.to(device)


writer = SummaryWriter(output_dir, flush_secs=10)
optimizer = torch.optim.SGD(model.parameters(), lr=lr,
                            momentum=momentum,
                            weight_decay=weight_decay)

milestones = [int(x) for x in lr_step]

scheduler = torch.optim.lr_scheduler.MultiStepLR(
    optimizer, milestones=milestones, gamma=0.1)
# HINT alter the scheduler to add learning rate warmup to your training. Consider creating
# another warmup_scheduler and using torch.optim.lr_scheduler.ChainedScheduler in order to chain your current and the warmup scheduler together.


optimizer.zero_grad()
dataloader_iter = None

cls_loss_np, bbox_loss_np, total_loss_np = [], [], []

pbar = tqdm(range(max_iter))

lossFunc = losses.LossFunc()


if not os.path.exists(output_dir):
    os.makedirs(output_dir)


# RELOAD LATEST CHECKPOINT FROM output_dir FOLDER

Useful for resuming training


In [ ]:
# load model weights from most recent file in checkpoint directory if it exists
import re


def load_latest_checkpoint(model, directory):
    """
    Load the latest checkpoint from a directory.

    :param directory: Directory containing the checkpoints of the form model{i}.pth
    :return: Loaded model checkpoint, iteration number
    """
    all_files = os.listdir(directory)

    # find all checkpoints of the form "model{i}.pth"
    checkpoint_files = [f for f in all_files if re.match(r"model_\d+\.pth", f)]

    if not checkpoint_files:
        raise ValueError("No valid checkpoint files found in the directory.")

    # Extract the iteration number 'i' from each filename
    iterations = [
        int(re.search(r"model_(\d+)\.pth", f).group(1)) for f in checkpoint_files
    ]

    # Find the file with the highest iteration number.
    max_iteration = max(iterations)
    latest_checkpoint_file = os.path.join(directory, f"model_{max_iteration}.pth")
    print(f"Loading model with checkpoint file {latest_checkpoint_file}")

    # Load the model with latest_checkpoint file
    model.load_state_dict(torch.load(latest_checkpoint_file))
    return model, max_iteration


model, checkpoint_iteration = load_latest_checkpoint(model, output_dir)

pbar = tqdm(range(max_iter))
pbar.update(checkpoint_iteration)

### Prediction and Validate Functions

(taken from predict.py)


In [ ]:
from detection_utils import (
    compute_targets,
    compute_bbox_targets,
    get_detections,
    apply_bbox_deltas,
    nms,
)
import json


test_nms_thresh = 0.5
test_set = "test"
test_score_thresh = 0.05
test_det_thresh = 100
test_model_checkpoint = 85000
model_dir = "runs/retina-net-basic"


def predict_image(image, image_id, resize_factor, model, cat_map):
    model.eval()
    results = []

    with torch.no_grad():
        outs = model(image)

        pred_clss, pred_bboxes, anchors = get_detections(outs)
        prob_clss = pred_clss.sigmoid()
        prob_clss = prob_clss[0, ...]
        pred_bboxes = pred_bboxes[0, ...]
        anchors = anchors[0, ...]
        out_bboxes = apply_bbox_deltas(anchors, pred_bboxes)
        out_bboxes = out_bboxes / resize_factor[0]

        for j in range(prob_clss.shape[1]):
            prob_cls = prob_clss[:, j]
            out_bbox = out_bboxes + 0.0
            anchor = anchors + 0.0
            keep = prob_cls > test_score_thresh
            prob_cls = prob_cls[keep]
            out_bbox = out_bbox[keep, :]
            anchor = anchor[keep, :]
            keep = nms(anchor, prob_cls, test_nms_thresh)
            prob_cls = prob_cls[keep]
            out_bbox = out_bbox[keep, :]
            anchor = anchor[keep, :]
            keep = torch.argsort(prob_cls, descending=True)[:test_det_thresh]
            prob_cls = prob_cls[keep]
            out_bbox = out_bbox[keep, :]
            anchor = anchor[keep, :]

            for a, p in zip(
                out_bbox.cpu().double().numpy(), prob_cls.cpu().double().numpy()
            ):
                res = {}
                res["image_id"] = int(image_id[0])
                res["category_id"] = cat_map.index(j + 1)
                res["bbox"] = [a[0], a[1], a[2] - a[0], a[3] - a[1]]
                res["score"] = p
                results.append(res)
    return results


def validate(dataset, dataloader, device, model, result_file_name, writer, iteration):
    model.eval()

    results = []
    for i, (image, _, _, _, image_id, resize_factor) in enumerate(tqdm(dataloader)):
        image = image.to(device)
        results += predict_image(
            image, image_id, resize_factor, model, list(dataset.cat_map)
        )
    if len(results) > 0:
        json.dump(results, open(result_file_name, "w"))
        metrics, classes = dataset.evaluate(result_file_name)
        print_results(metrics, classes)
        log_results(writer, metrics, classes, iteration)
    else:
        print(
            f"No detections above detection threshold of {test_score_thresh}, skipping evaluation."
        )

def test(dataset, dataloader, device, model, result_file_name):
    model.eval()
    
    results = []
    for i, (image, _, _, _, image_id, resize_factor) in enumerate(tqdm(dataloader)):
        image = image.to(device)
        results += predict_image(image, image_id, resize_factor, model, 
                                 list(dataset.cat_map))
    
    json.dump(results, open(result_file_name, 'w'))

def print_results(metrics, classes):
    tt = ""
    print(f"{tt:10s}\tAP\tAP50\tAP75\tAPs\tAPm\tAPl")
    for i, (m, c) in enumerate(zip(metrics, classes)):
        print(
            f"{c:>10s}\t{m[0]:.3f}\t{m[1]:.3f}\t{m[2]:.3f}\t{m[3]:.3f}\t{m[4]:.3f}\t{m[5]:.3f}"
        )


def log_results(writer, metrics, classes, iteration):
    for i, (m, c) in enumerate(zip(metrics, classes)):
        if i == 0:
            writer.add_scalar(f"AP", m[0], iteration)
            writer.add_scalar(f"AP50", m[1], iteration)
            writer.add_scalar(f"AP75", m[2], iteration)
            for j, tag in enumerate(["", "50", "75", "s", "m", "l"]):
                writer.add_scalar(f"bbox/AP{tag}", m[j] * 100, iteration)
        else:
            writer.add_scalar(f"AP/{c}", m[0], iteration)
            writer.add_scalar(f"AP50/{c}", m[1], iteration)
            writer.add_scalar(f"AP75/{c}", m[2], iteration)
            writer.add_scalar(f"bbox/AP-{c}", m[0] * 100, iteration)

## TRAINING LOOP

Feel free to alter how often we save the model's weights or evaluate on the validation data


In [ ]:
for i in pbar:
    if dataloader_iter is None or i % len(dataloader_iter) == 0:
        dataloader_iter = iter(dataloader_train)

    image, cls, bbox, is_crowd, image_id, _ = next(dataloader_iter)

    if len(bbox) == 0:
        continue

    image = image.to(device)
    bbox = bbox.to(device)
    cls = cls.to(device)

    outs = model(image)
    pred_clss, pred_bboxes, anchors = get_detections(outs)
    gt_clss, gt_bboxes = compute_targets(anchors, cls, bbox, is_crowd)

    pred_clss = pred_clss.sigmoid()
    classification_loss, regression_loss = lossFunc(
        pred_clss, pred_bboxes, anchors, gt_clss, gt_bboxes
    )
    cls_loss = classification_loss.mean()
    bbox_loss = regression_loss.mean()
    total_loss = cls_loss + bbox_loss

    if np.isnan(total_loss.item()):
        print("Loss went to NaN")
        break

    if np.isinf(total_loss.item()):
        print("Loss went to Inf")
        break

    total_loss.backward()

    # HINT: add gradient clipping here(it is only one line of code!)


    optimizer.step()
    optimizer.zero_grad()
    scheduler.step()

    # Some logging
    lr = scheduler.get_last_lr()[0]
    total_loss_np.append(total_loss.item())
    cls_loss_np.append(cls_loss.item())
    bbox_loss_np.append(bbox_loss.item())

    if (i + 1) % 20 == 0:
        writer.add_scalar("loss_box_reg", np.mean(bbox_loss_np), i + 1)
        writer.add_scalar("lr", lr, i + 1)
        writer.add_scalar("loss_cls", np.mean(cls_loss_np), i + 1)
        writer.add_scalar("total_loss", np.mean(total_loss_np), i + 1)
        pbar.set_description(
            f"{i+1} / {lr:5.6f} / {np.mean(cls_loss_np):5.3f} / {np.mean(bbox_loss_np):5.3f} / {np.mean(total_loss_np):5.3f}"
        )
        cls_loss_np, bbox_loss_np, total_loss_np = [], [], []

    if (i + 1) % 1000 == 0:
        torch.save(model.state_dict(), f"{output_dir}/model_{i+1}.pth")

    if (i + 1) % 5000 == 0 or (i + 1) == len(pbar):
        print("Validating...")
        val_dataloader = DataLoader(dataset_val, num_workers=3, collate_fn=collater)
        result_file_name = f"{output_dir}/results_{i+1}_val.json"
        model.eval()
        validate(
            dataset_val, val_dataloader, device, model, result_file_name, writer, i + 1
        )
        model.train()

torch.save(model.state_dict(), f"{output_dir}/model_final.pth")

# Save prediction result on test set
dataset_test = CocoDataset(
    "test", transform=transforms.Compose([Normalizer(), Resizer()])
)
test_dataloader = DataLoader(dataset_test, num_workers=1, collate_fn=collater)
result_file_name = f"{output_dir}/results_{max_iter}_test.json"
model.eval()
test(dataset_test, test_dataloader, device, model, result_file_name)

In [ ]:
# EVALUATE YOUR RESULTS ON THE TEST DATA (THIS IS WHAT YOU WILL SUBMIT TO GRADESCOPE)
iteration_num = 10000
!python predict.py --test_model_checkpoint {iteration_num} --test_set test --model_dir {output_dir}

## 7. Visualize Detection Results

**[1 pt, manually graded]**
For your best model, visualize some false positives and some false negatives. Describe any trends you see and include necessary supporting visualizations.
